# Importation

In [ ]:
import geopandas as gpd
from osgeo import gdal
import os
import numpy as np
import sys
sys.path.append('/home/onyxia/work/libsigma')
import read_and_write as rw

In [ ]:
def supprimer_dossier_non_vide(dossier):
    # Parcourir tout le contenu du dossier
    for element in os.listdir(dossier):
        chemin_element = os.path.join(dossier, element)
        # Vérifier si c'est un fichier
        if os.path.isfile(chemin_element) or os.path.islink(chemin_element):
            os.remove(chemin_element)  # Supprimer le fichier ou le lien
        elif os.path.isdir(chemin_element):
            supprimer_dossier_non_vide(chemin_element)  # Appel récursif pour les sous-dossiers
    # Supprimer le dossier une fois qu'il est vide
    os.rmdir(dossier)

# Raster 60 bandes : trop long pas assez de place sur le serv

In [ ]:
#  Définition des paramètres 

input_raster_dir = "/home/onyxia/work/data/images"
L_images = sorted(os.listdir(input_raster_dir))
L_images.remove('.keep')
ref_raster_path = "/home/onyxia/work/data/images/SENTINEL2B_20220125-105852-948_L2A_T31TCJ_C_V3-0_SRE_B2.tif"
shapefile_path = "/home/onyxia/work/data/project/emprise_etude.shp"
mask_foret_path = "/home/onyxia/Projet_Teledec/results/data/img_pretraitees/masque_foret.tif"
output_dir = "/home/onyxia/work/output"

os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Construction array

x,y = rw.get_image_dimension(rw.open_image(ref_raster_path))[:2]
bandes = 60
array_tot = np.zeros((x,y,bandes))

L_array = []
for i,img in enumerate(L_images):
    path = os.path.join(input_raster_dir,img) 
    array = rw.load_img_as_array(path)
    L_array.append(array)

In [ ]:
gdal.SetConfigOption("GDAL_CACHEMAX", "1000")  # Cache en Mo

In [ ]:
import numpy as np
from osgeo import gdal

def resample_to_resolution(src_array, src_transform, src_projection, target_resolution, target_shape):
    """
    Rééchantillonne un tableau numpy à une nouvelle résolution en utilisant GDAL.
    """
    mem_driver = gdal.GetDriverByName("MEM")
    target_rows, target_cols = target_shape

    # Créer une image GDAL en mémoire pour l'entrée
    src_ds = mem_driver.Create("", src_array.shape[1], src_array.shape[0], 1, gdal.GDT_Float32)
    src_ds.GetRasterBand(1).WriteArray(src_array)
    src_ds.SetGeoTransform(src_transform)
    src_ds.SetProjection(src_projection)

    # Créer une image GDAL en mémoire pour la sortie
    target_ds = mem_driver.Create("", target_cols, target_rows, 1, gdal.GDT_Byte, options=["COMPRESS=LZW", "TILED=YES"])
    target_transform = (
        src_transform[0], target_resolution, 0,
        src_transform[3], 0, -target_resolution
    )
    target_ds.SetGeoTransform(target_transform)
    target_ds.SetProjection(src_projection)

    # Créer les options de rééchantillonnage
    warp_options = gdal.WarpOptions(
        resampleAlg="bilinear",  # Algorithme bilinéaire
        multithread=True         # Utilisation de plusieurs threads pour accélérer
    )

    # Rééchantillonner avec GDAL Warp
    gdal.Warp(target_ds, src_ds, options=warp_options)
    
    # Lire et convertir en UInt8
    resampled_array = target_ds.GetRasterBand(1).ReadAsArray().astype(np.uint8)
    return resampled_array

def process_image(image_path, target_resolution, target_shape):
    """
    Charge et rééchantillonne une image entière.
    """
    ds = gdal.Open(image_path)
    array = ds.GetRasterBand(1).ReadAsArray()
    transform = ds.GetGeoTransform()
    projection = ds.GetProjection()

    resampled_array = resample_to_resolution(array, transform, projection, target_resolution, target_shape)
    return resampled_array

def concatenate_images(image_paths, target_resolution):
    """
    Concatène toutes les images après rééchantillonnage.
    """
    target_shape = None
    resampled_arrays = []

    for i, path in enumerate(image_paths):
        # Charger la première image pour calculer la forme cible
        if target_shape is None:
            ds = gdal.Open(path)
            transform = ds.GetGeoTransform()
            
            # Calculer les dimensions cibles
            cols = int((ds.RasterXSize * transform[1]) / target_resolution)
            rows = int((ds.RasterYSize * abs(transform[5])) / target_resolution)
            target_shape = (rows, cols)

        # Rééchantillonner chaque image
        resampled_array = process_image(path, target_resolution, target_shape)
        resampled_arrays.append(resampled_array)

        # Suivi de la progression
        print(f"Image {i+1}/{len(image_paths)} traitée")

    # Concaténer les tableaux après rééchantillonnage
    concatenated_array = np.stack(resampled_arrays, axis=0)
    return concatenated_array


# # Exemple d'utilisation
# image_paths = ["image1.tif", "image2.tif", "image3.tif"]  # Chemins vers les images Sentinel-2
# target_resolution = 10  # Résolution cible en mètres
# result = concatenate_images(image_paths, target_resolution)

In [ ]:
image_paths = []
for i in L_images:
    image_paths.append(os.path.join('/home/onyxia/work/data/images',i))
target_resolution = 10  # Résolution cible en mètres
output_file = os.path.join(output_dir,"Serie_temp_S2_allbands.tif")  # Nom de fichier de sortie

In [ ]:
dataset_exist = rw.open_image(image_paths[2])
rw.get_image_dimension(dataset_exist)

In [ ]:
#Def d'un dataset vierge pour l'écriture
driver = gdal.GetDriverByName('GTiff')

out_filename = 'dataset_vierge.tiff'
nb_col,nb_ligne,nb_band = rw.get_image_dimension(dataset_exist)
gdal_dtype = gdal.GDT_Byte
output_data_set = driver.Create(out_filename, nb_col, nb_ligne, nb_band, gdal_dtype)

#Ouverture de ce dataset
dataset = rw.open_image('/home/onyxia/work/Projet_Teledec/scripts/dataset_vierge.tiff')

In [ ]:
result = concatenate_images(image_paths, target_resolution)

In [ ]:
dataset = rw.open_image(ref_raster_path)
out_path=os.path.join(output_dir,"Serie_temp_S2_allbands.tif")
rw.write_image(out_filename = out_path, array = result, gdal_dtype = gdal.GDT_Byte, data_set = dataset)

In [ ]:
os.rmdir(output_dir)

```
Je vais essayer de faire les prétraitements en créant des fichiers intermédiaires
En commençant par le clip
Resolution 10m (faire un if pour faire le traitement si resol != 10m)
Projection
Encodé uint8
No data 0
Non foret masquée
60 fois
```

# Technique alternative

## Version itérative

In [ ]:
#  Définition des paramètres 

input_raster_dir = "/home/onyxia/work/data/images"
L_images = sorted(os.listdir(input_raster_dir))
if ".keep" in L_images:
    L_images.remove(".keep")
shapefile_path = "/home/onyxia/work/data/project/emprise_etude.shp"

output_dir = "/home/onyxia/work/output"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
#Façon pour clip/resol/nodata/format/proj un raster grâce à l'emprise

# Charger le vecteur avec Geopandas
emprise = gpd.read_file(shapefile_path).to_crs("EPSG:2154")

# Extraire le GeoJSON sous forme de string
geojson_str = emprise.to_json()

for i,img in enumerate(L_images) :
    date = img[11:19]
    bande = img[53:]
    ds_img = rw.open_image(os.path.join(input_raster_dir,img))
    name_file = f"traitement_{date}_{bande}"
    output_file = os.path.join(output_dir,name_file)
    # Appliquer le clip avec GDAL
    resolution = 10  # Résolution (10 m)
    output_raster_test = gdal.Warp(
        output_file, # Chemin de fichier, car on utilise GTiff
        # "",  # Pas de chemin de fichier, car on utilise MEM
        ds_img,  # Fichier en entrée (chemin ou objet gdal)
        format = "GTiff", # Utiliser GTiff comme format
        # format = "MEM",  # Utiliser MEM comme format
        cutlineDSName = geojson_str,  # Passer directement le GeoJSON
        cropToCutline = True,
        outputType = gdal.GDT_UInt16, #UInt16
        dstSRS = "EPSG:2154",  # Reprojection
        xRes = resolution,  # Résolution X
        yRes = resolution,  # Résolution Y
        dstNodata = 0  # Valeur NoData
    )
    print(f"Image {i+1}/{len(L_images)} traitée")
ds_img = None
emprise = None
geojson_str = None

In [ ]:
# Construction array
ref_raster_path = "/home/onyxia/work/output/traitement_20220125_B2.tif"
L_images_clip = sorted(os.listdir(output_dir))
x,y = rw.get_image_dimension(rw.open_image(ref_raster_path))[:2]
bandes = 60
array_tot = np.zeros((x,y,bandes))

L_array = []
for i,img in enumerate(L_images_clip):
    path = os.path.join(output_dir,img) 
    array = rw.load_img_as_array(path)
    L_array.append(array)

# Concat array
print("Concaténation en cours")
array_final = np.concatenate(L_array,axis = 2)
print("Tableau concaténé")

# Save array into image
out = "/home/onyxia/work/Projet_Teledec/results/data/img_pretraitees/Serie_temp_S2_allbands.tif"
print("Ecriture en cours")
rw.write_image(out_filename=out, array = array_final, data_set = rw.open_image(ref_raster_path))
print("Ecriture terminée")

In [ ]:
#Supp du dossier output pour faire de la place

supprimer_dossier_non_vide("/home/onyxia/work/output")